In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [2]:
df = pd.read_csv("data.csv")

In [3]:
df.head(5)

,IM_INCIDENT_KEY,FIRE_BOX,INCIDENT_TYPE_DESC,INCIDENT_DATE_TIME,ARRIVAL_DATE_TIME,UNITS_ONSCENE,LAST_UNIT_CLEARED_DATE_TIME,HIGHEST_LEVEL_DESC,TOTAL_INCIDENT_DURATION,ACTION_TAKEN1_DESC,...,ZIP_CODE,BOROUGH_DESC,FLOOR,CO_DETECTOR_PRESENT_DESC,FIRE_ORIGIN_BELOW_GRADE_FLAG,STORY_FIRE_ORIGIN_COUNT,FIRE_SPREAD_DESC,DETECTOR_PRESENCE_DESC,AES_PRESENCE_DESC,STANDPIPE_SYS_PRESENT_FLAG
0,63476760,5535.0,412 - Gas leak (natural gas or LPG),06/30/2018 11:52:35 PM,NaN,NaN,07/01/2018 12:11:36 AM,11 - First Alarm,1141,44 - Hazardous materials leak control & contai...,...,11427.0,5 - Queens,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,63476693,2870.0,412 - Gas leak (natural gas or LPG),06/30/2018 11:35:27 PM,06/30/2018 11:40:19 PM,3.0,06/30/2018 11:50:45 PM,11 - First Alarm,918,216 - Checked exposures,...,10462.0,2 - Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,63476614,2894.0,412 - Gas leak (natural gas or LPG),06/30/2018 11:22:51 PM,06/30/2018 11:27:18 PM,1.0,06/30/2018 11:39:26 PM,11 - First Alarm,995,86 - Investigate,...,10472.0,2 - Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,63476562,2723.0,412 - Gas leak (natural gas or LPG),06/30/2018 11:05:55 PM,06/30/2018 11:11:39 PM,3.0,06/30/2018 11:17:06 PM,11 - First Alarm,671,44 - Hazardous materials leak control & contai...,...,10452.0,2 - Bronx,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,63476510,3729.0,412 - Gas leak (natural gas or LPG),06/30/2018 10:53:15 PM,06/30/2018 10:58:49 PM,4.0,06/30/2018 11:16:19 PM,11 - First Alarm,1384,44 - Hazardous materials leak control & contai...,...,10463.0,2 - Bronx,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119324 entries, 0 to 119323
Data columns (total 24 columns):
IM_INCIDENT_KEY                 119324 non-null int64
FIRE_BOX                        119322 non-null float64
INCIDENT_TYPE_DESC              119324 non-null object
INCIDENT_DATE_TIME              119324 non-null object
ARRIVAL_DATE_TIME               118636 non-null object
UNITS_ONSCENE                   118636 non-null float64
LAST_UNIT_CLEARED_DATE_TIME     119324 non-null object
HIGHEST_LEVEL_DESC              119322 non-null object
TOTAL_INCIDENT_DURATION         119324 non-null int64
ACTION_TAKEN1_DESC              119324 non-null object
ACTION_TAKEN2_DESC              115132 non-null object
ACTION_TAKEN3_DESC              113241 non-null object
PROPERTY_USE_DESC               119323 non-null object
STREET_HIGHWAY                  119319 non-null object
ZIP_CODE                        119322 non-null float64
BOROUGH_DESC                    119324 non-null object
FLOOR   

In [5]:
df.dtypes

IM_INCIDENT_KEY                   int64
FIRE_BOX                        float64
INCIDENT_TYPE_DESC               object
INCIDENT_DATE_TIME               object
ARRIVAL_DATE_TIME                object
UNITS_ONSCENE                   float64
LAST_UNIT_CLEARED_DATE_TIME      object
HIGHEST_LEVEL_DESC               object
TOTAL_INCIDENT_DURATION           int64
ACTION_TAKEN1_DESC               object
ACTION_TAKEN2_DESC               object
ACTION_TAKEN3_DESC               object
PROPERTY_USE_DESC                object
STREET_HIGHWAY                   object
ZIP_CODE                        float64
BOROUGH_DESC                     object
FLOOR                            object
CO_DETECTOR_PRESENT_DESC         object
FIRE_ORIGIN_BELOW_GRADE_FLAG    float64
STORY_FIRE_ORIGIN_COUNT         float64
FIRE_SPREAD_DESC                float64
DETECTOR_PRESENCE_DESC          float64
AES_PRESENCE_DESC               float64
STANDPIPE_SYS_PRESENT_FLAG      float64
dtype: object

In [6]:
## Remove unwanted columns

new_df = df[['IM_INCIDENT_KEY', 'INCIDENT_TYPE_DESC', 'INCIDENT_DATE_TIME', 'ARRIVAL_DATE_TIME','TOTAL_INCIDENT_DURATION',
             'ACTION_TAKEN1_DESC','ACTION_TAKEN2_DESC','ZIP_CODE','BOROUGH_DESC']]

In [7]:
new_df.head()

,IM_INCIDENT_KEY,INCIDENT_TYPE_DESC,INCIDENT_DATE_TIME,ARRIVAL_DATE_TIME,TOTAL_INCIDENT_DURATION,ACTION_TAKEN1_DESC,ACTION_TAKEN2_DESC,ZIP_CODE,BOROUGH_DESC
0,63476760,412 - Gas leak (natural gas or LPG),06/30/2018 11:52:35 PM,NaN,1141,44 - Hazardous materials leak control & contai...,64 - Shut down system,11427.0,5 - Queens
1,63476693,412 - Gas leak (natural gas or LPG),06/30/2018 11:35:27 PM,06/30/2018 11:40:19 PM,918,216 - Checked exposures,NaN,10462.0,2 - Bronx
2,63476614,412 - Gas leak (natural gas or LPG),06/30/2018 11:22:51 PM,06/30/2018 11:27:18 PM,995,86 - Investigate,45 - Remove hazard,10472.0,2 - Bronx
3,63476562,412 - Gas leak (natural gas or LPG),06/30/2018 11:05:55 PM,06/30/2018 11:11:39 PM,671,44 - Hazardous materials leak control & contai...,64 - Shut down system,10452.0,2 - Bronx
4,63476510,412 - Gas leak (natural gas or LPG),06/30/2018 10:53:15 PM,06/30/2018 10:58:49 PM,1384,44 - Hazardous materials leak control & contai...,64 - Shut down system,10463.0,2 - Bronx


In [8]:
## remove rows with blanks

new_df = new_df.dropna()

In [9]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114523 entries, 2 to 119323
Data columns (total 9 columns):
IM_INCIDENT_KEY            114523 non-null int64
INCIDENT_TYPE_DESC         114523 non-null object
INCIDENT_DATE_TIME         114523 non-null object
ARRIVAL_DATE_TIME          114523 non-null object
TOTAL_INCIDENT_DURATION    114523 non-null int64
ACTION_TAKEN1_DESC         114523 non-null object
ACTION_TAKEN2_DESC         114523 non-null object
ZIP_CODE                   114523 non-null float64
BOROUGH_DESC               114523 non-null object
dtypes: float64(1), int64(2), object(6)
memory usage: 8.7+ MB


In [10]:
## Change zip-code to INT to remove decimal

new_df['ZIP_CODE'] = new_df['ZIP_CODE'].astype(int)

In [11]:
new_df.head(3)

,IM_INCIDENT_KEY,INCIDENT_TYPE_DESC,INCIDENT_DATE_TIME,ARRIVAL_DATE_TIME,TOTAL_INCIDENT_DURATION,ACTION_TAKEN1_DESC,ACTION_TAKEN2_DESC,ZIP_CODE,BOROUGH_DESC
2,63476614,412 - Gas leak (natural gas or LPG),06/30/2018 11:22:51 PM,06/30/2018 11:27:18 PM,995,86 - Investigate,45 - Remove hazard,10472,2 - Bronx
3,63476562,412 - Gas leak (natural gas or LPG),06/30/2018 11:05:55 PM,06/30/2018 11:11:39 PM,671,44 - Hazardous materials leak control & contai...,64 - Shut down system,10452,2 - Bronx
4,63476510,412 - Gas leak (natural gas or LPG),06/30/2018 10:53:15 PM,06/30/2018 10:58:49 PM,1384,44 - Hazardous materials leak control & contai...,64 - Shut down system,10463,2 - Bronx


In [12]:
## Remove number associated with borough

new_df['BOROUGH_DESC'] = new_df['BOROUGH_DESC'].map(lambda x: str(x)[4:])

In [13]:
new_df.head(3)

,IM_INCIDENT_KEY,INCIDENT_TYPE_DESC,INCIDENT_DATE_TIME,ARRIVAL_DATE_TIME,TOTAL_INCIDENT_DURATION,ACTION_TAKEN1_DESC,ACTION_TAKEN2_DESC,ZIP_CODE,BOROUGH_DESC
2,63476614,412 - Gas leak (natural gas or LPG),06/30/2018 11:22:51 PM,06/30/2018 11:27:18 PM,995,86 - Investigate,45 - Remove hazard,10472,Bronx
3,63476562,412 - Gas leak (natural gas or LPG),06/30/2018 11:05:55 PM,06/30/2018 11:11:39 PM,671,44 - Hazardous materials leak control & contai...,64 - Shut down system,10452,Bronx
4,63476510,412 - Gas leak (natural gas or LPG),06/30/2018 10:53:15 PM,06/30/2018 10:58:49 PM,1384,44 - Hazardous materials leak control & contai...,64 - Shut down system,10463,Bronx


In [14]:
new_df.dtypes

IM_INCIDENT_KEY             int64
INCIDENT_TYPE_DESC         object
INCIDENT_DATE_TIME         object
ARRIVAL_DATE_TIME          object
TOTAL_INCIDENT_DURATION     int64
ACTION_TAKEN1_DESC         object
ACTION_TAKEN2_DESC         object
ZIP_CODE                    int64
BOROUGH_DESC               object
dtype: object

In [15]:
new_df.to_csv("coned_leaks.csv")